In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/FolloweeIDs2_tweets_df_AugustPull.csv")

/tmp/ipykernel_48725/2792471165.py:1: DtypeWarning: Columns (2,3,6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/FolloweeIDs2_tweets_df_AugustPull.csv")


In [3]:
df = df[df["reply_to_id"].astype(str) == "nan"]
df = df[df["retweeted_user_ID"].astype(str) == "nan"]
df = df[df["reply_to_user"].astype(str) == "nan"]
df["created_at"] = pd.to_datetime(df["created_at"])
df = df[(df["created_at"] > "2023-08-13") & (df["created_at"] <= "2023-08-15")]
df = df[df["full_text"].str.contains("http|www") == False]

In [4]:
# format created_at into 144 10-minute intervals on both days
df["created_at_interval"] = ((df["created_at"].dt.hour * 60 + df["created_at"].dt.minute) // 10).astype(int)

df["created_at_timestamp"] = df["created_at_interval"] + np.where(
    df["created_at"].dt.date == pd.to_datetime("2023-08-14").date(), 
    144, 
    0
)

In [5]:
feed = []

for idx, row in df.iterrows():
    feed.append({
        "user": {"id": row["screen_name"]},
        "content": row["full_text"],
        "id": idx,
        "timestamp": int(row["created_at_timestamp"]),
        "reads": [],
        "likes": []
    })

In [6]:
import json
with open("../data/snapshot.json", "w") as f:
    json.dump(feed, f)